# Pitch Outcome Transfer 
- using the pitch clusters from 3.1, train a general model
- can then use these as initial weights for models, which hopefully helps with low AB players.
## Todo:
- write query to get training data for each cluster
- train model using all of this data
- save weights
- write script to load the initial weights based on new batters cluster
- train model using this
- compare results

In [32]:
from importlib import reload

import torch
import torch.nn as nn
import torch.optim
import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
from tqdm import trange

#vladdy: 665489
#soto: 665742
#schneider 676914

#get test set
from src.features import build_features as f
reload(f)
#train_set, val_set, num_classes, num_features, encoder = f.get_pitch_outcome_dataset(665742)
train_set, val_set, num_classes, num_features, encoder = f.get_pitch_outcome_dataset_general(8, 'L')

2024-05-07 16:22:31,147 - src.features.build_features - INFO - Loading dataset for cluster 8
2024-05-07 16:22:37,676 - src.features.build_features - INFO - Data successfully queried/transformed for cluster 8


In [35]:
import torch.optim.lr_scheduler as lr_scheduler

In [20]:
class PitchOutcome(nn.Module):
    def __init__(self, input_size, num_classes):
        super(PitchOutcome, self).__init__()
        self.l1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(128, 64)
        self.l3 = nn.Linear(64, num_classes)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        return out

In [37]:
num_epochs = 50

input_size = num_features
num_classes = num_classes

model = PitchOutcome(input_size, num_classes)

loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)

scheduler = lr_scheduler.LinearLR(optim, start_factor=1.0, end_factor=0.3, total_iters=10)

val_accuracies, val_losses, train_losses, logliks = [], [], [], []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = 'cpu'
model.to(device)

for epoch in (t := trange(num_epochs)):

    #train model
    model.train()
    running_loss = 0    
    for features, labels in train_set:

        features, labels = features.to(device), labels.to(device)
        
        #zero grads
        optim.zero_grad()
        #forward + backward + optimize
        outputs = model(features)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optim.step()
        #track loss
        running_loss += loss.item()
    
    running_loss /= len(train_set)
    train_losses.append(running_loss)
    scheduler.step()


    #validate model
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    predicted_probs = []
    true_labels = []
    with torch.no_grad():
        for features, labels in val_set:
            #one_hot_labels = labels
            features, labels = features.to(device), labels.to(device)
            
            outputs = model(features)
            loss = loss_fn(outputs, labels)
            val_loss += loss.item()

            #track loglik
            predicted_probs.extend(torch.softmax(outputs, dim=1).cpu().numpy())  # Convert to probabilities
            true_labels.extend(labels.cpu().numpy())
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            #convert one hot back to label
            labels = torch.argmax(labels, dim=1)
            #print(predicted.shape, labels.shape, labels)
            correct += (predicted == labels).sum().item()
    
    val_loss /= len(val_set)
    val_losses.append(val_loss)
    val_accuracies.append(100 * correct//total)

    # Convert lists to numpy arrays
    predicted_probs = np.array(predicted_probs)
    log_predicted_probs = np.log(predicted_probs + 1e-10)
    true_labels = np.array(true_labels)
    
    #compute loglik for model
    true_class_indices = np.argmax(true_labels, axis=1)
    log_liks = log_predicted_probs[np.arange(len(true_class_indices)), true_class_indices]
    total_log_lik = np.sum(log_liks)
    logliks.append(total_log_lik)
    
    t.set_description(f'val_loglik: {total_log_lik:.4f} loss: {running_loss:.4f} val_accuracy: {100 * correct//total}%')

val_loglik: -46148.9570 loss: 0.8863 val_accuracy: 60%: 100%|█████████████████████████| 50/50 [04:57<00:00,  5.96s/it]


In [38]:
def pitch_outcome_loglik(model, train_set, val_set):

    train_true_labels = []
    predicted_probs = []
    true_labels = []
    
    # Set model to evaluation mode
    model.to('cpu')
    model.eval()
    
    # Get predicted probabilities and true labels
    with torch.no_grad():
        for features, labels in val_set:
            features, labels = features, labels
            outputs = model(features)
            predicted_probs.extend(torch.softmax(outputs, dim=1).numpy())  # Convert to probabilities
            true_labels.extend(labels.numpy())
            train_true_labels.extend(labels.numpy())

    # Convert lists to numpy arrays
    predicted_probs = np.array(predicted_probs)
    print(np.max(predicted_probs,axis=0))
    log_predicted_probs = np.log(predicted_probs +1e-10)
    true_labels = np.array(true_labels)
    train_true_labels = np.array(train_true_labels)
    
    #compute loglik for model
    true_class_indices = np.argmax(true_labels, axis=1)
    log_liks = log_predicted_probs[np.arange(len(true_class_indices)), true_class_indices]
    total_log_lik = np.sum(log_liks)

    #------------- val loglik is better than multinomail ----------
    #compute loglik for multinomial model
    
    #computes proportion of times a certain value showed up
    categorical_p_est = np.mean(train_true_labels, axis=0)
    log_predicted_probs = np.log(categorical_p_est)
    categorical_dist_logliks = log_predicted_probs[true_class_indices]
    categorical_log_lik = np.sum(categorical_dist_logliks)

    print(f'ML Pitch Outcome Model LogLik: {total_log_lik:.2f}\nStandard Categorical Distribution LogLik: {categorical_log_lik:.2f}')

    assert(total_log_lik > categorical_log_lik)
    
    return total_log_lik, categorical_log_lik


pitch_outcome_loglik(model, train_set, val_set)

[1.        0.6137584 0.8662361 0.5982355 1.       ]
ML Pitch Outcome Model LogLik: -46148.95
Standard Categorical Distribution LogLik: -68266.27


(-46148.953, -68266.27)

In [17]:
# model which doesn't account for batter stance:
#loglik: -77581.53 vs -112017.59
no_stance = -77581.53 / -112017.59

0.6925834594370402

In [12]:
print(a:=torch.eye(5))
encoder['pitch_outcome'].inverse_transform(a)

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])


array([['ball'],
       ['foul'],
       ['hit_by_pitch'],
       ['hit_into_play'],
       ['strike']], dtype=object)

In [39]:
torch.save(model.state_dict(), '../models/pitch_outcome/cluster_models/cluster_8_bats_L.pth')